# Jupyter Notebook: Klassifikation mit einem Multi Layer Perceptron in PyTorch

In diesem Notebook werden wir ein **Multi Layer Perceptron (MLP)** mit **PyTorch** erstellen, um eine Klassifikationsaufgabe auf tabellarischen Daten durchzuführen. Wir verwenden den **Adult Income Dataset**, der demografische Informationen enthält und die Aufgabe stellt, vorherzusagen, ob eine Person ein Einkommen über oder unter 50.000 USD hat. Dies ist eine binäre Klassifikationsaufgabe.

Das Notebook ist in folgende Abschnitte unterteilt:
1. **Datenvorbereitung**: Laden, Bereinigen und Vorbereiten des Datensatzes.
2. **Erstellen von PyTorch Datasets und Dataloaders**: Umwandeln der Daten in ein für PyTorch geeignetes Format.
3. **Definition des MLP-Modells**: Erstellen eines einfachen MLP mit einer versteckten Schicht.
4. **Training des Modells**: Verwenden des Adam-Optimizers und der Cross-Entropy-Loss-Funktion.
5. **Evaluation des Modells**: Berechnen von Metriken wie Accuracy, Precision, Recall und F1-Score sowie Visualisierung der Confusion Matrix.

Jeder Abschnitt enthält Erklärungen und Code, um den Studierenden ein tiefes Verständnis der Konzepte zu vermitteln.

## 1. Datenvorbereitung

Zuerst müssen wir den Datensatz laden und vorbereiten. Der **Adult Income Dataset** enthält sowohl numerische als auch kategorische Merkmale, die wir verarbeiten müssen.

### 1.1 Datensatz laden

Der Datensatz kann von [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/adult) heruntergeladen werden. Für dieses Beispiel laden wir ihn direkt aus dem Internet.

In [ ]:
import pandas as pd

# URL des Datensatzes
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

# Spaltennamen
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

# Daten laden
data = pd.read_csv(url, header=None, names=columns, na_values=" ?", skipinitialspace=True)

# Erste Zeilen anzeigen
print(data.head())

### 1.2 Daten bereinigen

Wir entfernen Zeilen mit fehlenden Werten und kodieren kategorische Variablen in numerische Werte.

In [ ]:
# Fehlende Werte entfernen
data = data.dropna()

# Kategorische Variablen in numerische umwandeln
from sklearn.preprocessing import LabelEncoder

categorical_cols = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country", "income"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Zielvariable (income) extrahieren
X = data.drop("income", axis=1)
y = data["income"]

### 1.3 Daten in Trainings-, Validierungs- und Testsets aufteilen

Wir teilen den Datensatz in 70% Training, 15% Validierung und 15% Test.

In [ ]:
from sklearn.model_selection import train_test_split

# Aufteilung in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Aufteilung des Trainingsdatensatzes in Trainings- und Validierungsdaten
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1765, random_state=42)  # 0.1765 * 85% ≈ 15%

### 1.4 Daten skalieren

Da neuronale Netze empfindlich auf die Skalierung der Eingabedaten reagieren, standardisieren wir die numerischen Merkmale.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## 2. Erstellen von PyTorch Datasets und Dataloaders

PyTorch arbeitet mit **Datasets** und **Dataloaders**, um Daten effizient zu laden und zu verarbeiten.

### 2.1 Benutzerdefiniertes Dataset erstellen

Wir erstellen ein benutzerdefiniertes Dataset, das die Daten in Tensoren umwandelt.

### 2.2 Dataloaders erstellen

Dataloaders ermöglichen das Laden der Daten in Batches und das Mischen der Daten.

## 3. Definition des MLP-Modells

Ein **Multi Layer Perceptron (MLP)** besteht aus einer Eingabeschicht, einer oder mehreren versteckten Schichten und einer Ausgabeschicht. Für dieses Beispiel verwenden wir ein einfaches MLP mit einer versteckten Schicht.

### 3.1 Modellarchitektur

- **Eingabeschicht**: Die Anzahl der Neuronen entspricht der Anzahl der Merkmale (Features) im Datensatz.
- **Versteckte Schicht**: Wir wählen 64 Neuronen mit ReLU-Aktivierungsfunktion.
- **Ausgabeschicht**: 2 Neuronen (für die binäre Klassifikation) mit Softmax-Aktivierungsfunktion.

## 4. Training des Modells

Wir trainieren das Modell mit dem **Adam-Optimizer** und der **Cross-Entropy-Loss-Funktion**, die für Klassifikationsaufgaben geeignet ist.

### 4.1 Funktion zum Plotten der Verluste

Wir definieren eine Funktion, die die Trainings- und Validierungsverluste über die Epochen hinweg speichert und einen Plot erstellt.

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(train_losses, val_losses, num_epochs):
    """
    Plottet die Trainings- und Validierungsverluste über die Epochen.

    Args:
        train_losses (list): Liste der durchschnittlichen Trainingsverluste pro Epoche
        val_losses (list): Liste der durchschnittlichen Validierungsverluste pro Epoche
        num_epochs (int): Anzahl der Epochen
    """
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Trainings- und Validierungsverluste über Epochen')
    plt.legend()
    plt.grid(True)
    plt.show()

### 4.2 Trainingsschleife

Wir trainieren das Modell für eine bestimmte Anzahl von Epochen und überwachen die Leistung auf dem Validierungsdatensatz.

## 5. Evaluation des Modells

Nach dem Training evaluieren wir das Modell auf dem Testdatensatz mit gängigen Metriken wie **Accuracy**, **Precision**, **Recall** und **F1-Score**. Zusätzlich visualisieren wir die **Confusion Matrix**.

### 5.1 Metriken berechnen

Wir verwenden die Funktionen von `sklearn` zur Berechnung der Metriken.

### 5.2 Confusion Matrix visualisieren

Die Confusion Matrix hilft uns, die Vorhersagen des Modells besser zu verstehen.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Confusion Matrix berechnen
cm = confusion_matrix(all_labels, all_preds)

# Plotten
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["<=50K", ">50K"], yticklabels=["<=50K", ">50K"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()